# 朴素贝叶斯（Naive Bayes）
### 原理
朴素贝叶斯方法是贝叶斯决策理论的一部分。贝叶斯决策理论的核心思想即选择具有最高概率的决策。
朴素贝叶斯法是基于贝叶斯定理和特征条件独立假设的分类方法。对于给定的训练数据集，首先基于特征条件独立假设学习输入/输出的联合概率分布；然后基于此模型，对给定的输入x，利用贝叶斯定理求出后验概率最大的输出y。
### 朴素
之所谓称为朴素，是因为整个形式化过程只做最原始、最简单的假设。
朴素贝叶斯分类器的一个假设是：假设特征之间相互独立，所谓独立指的是统计意义上的独立，即一个特征或者单词出现的可能性与它和其他单词相邻没有关系。
另一个假设是：每个特征同等重要。
### 优缺点
朴素贝叶斯法实现简单，学习和预测的效率都很高，是一种常用的方法。
在数据较少的情况下仍然有效，可以处理多类别问题。
对于输入数据的准备方式比较敏感

### 其他的几个重要概念
- 概率（Probability）
0和1之间的一个数字，表示一个特定结果发生的可能性。比如投资硬币，“正面朝上”这个特定结果发生的可能性为0.5，这个0.5就是概率。换一种说法，计算样本数据中出现该结果次数的百分比。即你投一百次硬币，正面朝上的次数基本上是50次。
- 几率（Odds）
某一特定结果发生与不发生的概率比。如果你明天电梯上遇上你暗恋的女孩的概率是0.1，那么遇不上她的概率就是0.9，那么遇上暗恋女孩的几率就是1/9，几率的取值范围是0到无穷大。
- 似然（Likelihood）
两个相关的条件概率之比，即给定B发生的情况下，某一特定结果A发生的概率和给定B不发生的情况下A发生的概率之比。另一种表达方式是，给定B的情况下A发生的几率和A的整体几率之比。两个计算方式是等价的。
## 条件概率
在B事件发生的基础上发生A的概率表示为P(A|B)

### 贝叶斯公式
P(A|B)=P(AB)/P(B)=P(B|A)* P(A)/P(B)
从公式中可知，如果要计算B条件下A发生的概率，只需要计算出后面等式的三个部分，B事件的概率（P(B)），是B的先验概率、A属于某类的概率（P(A)），是A的先验概率、以及已知A的某个分类下，事件B的概率（P(B|A)），是后验概率。
如果要确定某个样本归属于哪一类，则需要计算出归属不同类的概率，再从中挑选出最大的概率
我们把上面的贝叶斯公式写出这样，也许你能更好的理解：
MAX(P(Ai|B))=MAX(P(B|Ai)* P(Ai)/P(B))
## 步骤：
1）确定特征属性，获取样本数据；
2）对每个类别计算p(Ci)
3）对每个特征计算所划分的条件概率p(xj|Ci)
4）对每各类别计算p(x|Ci)p(Ci)
5）选择p(x|Ci)p(Ci)最大的Ci作为x的类
## 概率的计算采用贝叶斯公式
$$p(c_{i}|W)=\frac{p(W|c_{i})p(c_{i})}{p(W)}$$
又根据朴素假设，所有词汇概率统计独立
$$p(W|c_{i})=p(w_{0},w_{1},w_{2},...,w_{n}|c_{i})=p(w_{0}|c_{i})\cdot p(w_{1}|c_{i})\cdot p(w_{2}|c_{i})\cdot ...\cdot p(w_{n}|c_{i})$$

# 使用朴素贝叶斯过滤垃圾邮件
**说明:**
将 `email` 文件夹放在当前目录下。

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import log
#%matplotlib inline 可以在Ipython编译器里直接使用，功能是可以内嵌绘图，并且可以省略掉plt.show()这一步
%matplotlib inline

## 4.1.文本分类

In [ ]:
#词表到向量的转换函数

In [2]:
def loadDataSet():
    """
    生成一个文本数据集和标签
    参数：
        无
    返回：
        postingList -- 文本列表
        classVec -- 标签分类
    
    """
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]                    #0-非侮辱性，1-侮辱性
    return postingList,classVec

In [3]:
def createVocabList(dataSet):
    """
    建立词汇表，也就是所有文本的并集（一个包含所有文档中出现的不重复词的列表）
    参数：
        dataSet -- 数据集
    返回：
        vocabSet -- 词汇表
    """
    vocabSet = set([])                       #首先建立一个空集，新建一个集合，避免直接修改原数据
    for document in dataSet:                 #遍历数据集中的每个文档
        vocabSet = vocabSet | set(document)  #每个文档和之前的词汇表求并集，保证没有重复
    return list(vocabSet)

In [17]:
#文档词集模型（将每个词出现与否作为一个特征）
def setOfWords2Vec(vocabList, inputSet):
    """
    用词汇表向量表达句子
    向量中为1，代表词汇表对应该处的词汇；反之，不对应该处词汇
    参数：
        vocabList -- 词汇表
        inputSet --  单个文档
    返回：
        returnVec -- 返回句子对应的特征向量
    """  
    returnVec = [0]*len(vocabList) #初始化返回句子特征向量，长度为词汇表长度，全0向量

    for word in inputSet:                         #遍历该文档
        if word in vocabList:                     #如果该词汇在词汇表中      
            returnVec[vocabList.index(word)] = 1  #对应序号元素置1        
        else:                                     #反之给出错误提示
            print("词汇“{}”不在词汇表中".format(word))
    return returnVec
#文档的词袋模型（考虑词条的出现次数）
def bagOfWords2VecMN(vocabList, inputSet):
    """
    文档的词袋模型
    参数：
        vocabList -- 词汇列表
        inputSet -- 测试的单个文档
    返回：
        returnVec -- 返回向量
    """
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            #如果在词库中，对应词条就++
            returnVec[vocabList.index(word)] += 1
    return returnVec

In [5]:
def trainNB0(trainMatrix,trainCategory):
    """
    训练朴素贝叶斯分类器
    参数：
        trainMatrix --   输入的训练矩阵
        trainCategory -- 训练标签
    返回：
        p0Vect -- 非侮辱性类别条件下，每个词条概率组成的向量
        p1Vect -- 侮辱性类别条件下，每个词条的概率组成的向量
        pAbusive -- 侮辱性文档的概率
    """ 
    trainData = np.array(trainMatrix)                 #将数据转换为矩阵形式
    trainLabels = np.array(trainCategory)    
    numTrainDocs = trainData.shape[0]                 #训练文档数
    numWords = trainData.shape[1]                     #训练词条数
    pAbusive = np.mean(trainLabels)                   #侮辱性文本的先验概率    
    p0Num = np.ones((1, numWords))                    #所有词出现数初始化为全1避免最后乘积为0
    p1Num = np.ones((1, numWords))  
    p0Denom = 2.0; p1Denom = 2.0                      #分母初始化为2
                                       #Bayes分类只用比较概率的相对大小，所以这么做不会影响结果
    #p1Num为给定侮辱文档类别下统计每个侮辱性词条出现次数的向量，等于文本中标记1的子矩阵，沿axis = 0相加
    p1Num += np.sum(trainData[trainLabels == 1, :], axis = 0, keepdims = True)
    #p1Denom为给定侮辱文档类别下统计侮辱性词条总数的标量，等于文本中标记1的子矩阵向量之和
    p1Denom += np.sum(trainData[trainLabels == 1, :])
    #p0Num为给定非侮辱文档类别下统计每个侮辱性词条出现次数的向量，等于文本中标记1的子矩阵，沿axis = 0相加
    p0Num += np.sum(trainData[trainLabels == 0, :], axis = 0, keepdims = True)
    #p0Denom为给定侮辱文档类别下统计侮辱性词条总数的标量，等于文本中标记1的子矩阵向量之和
    p0Denom += np.sum(trainData[trainLabels == 0, :])
    #对概率取对数，可以避免下溢出或者浮点数舍入误差，同样由于相对大小不变，不影响结果
    p1Vect = np.log(p1Num/p1Denom)
    p0Vect = np.log(p0Num/p0Denom)
    return p0Vect,p1Vect,pAbusive

In [6]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    """
    计算后验概率，比较概率，得出结果
    由于所有概率均取了对数，所以乘法变加法，除法变减法
    参数：
        vec2Classify -- 要分类的文本向量
        p0Vec -- 给定非侮辱文档类别下统计每个侮辱性词条出现次数的向量  （类别0的似然函数（标称型数据））
        p1Vec -- 给定侮辱文档类别下统计每个侮辱性词条出现次数的向量    （类别1的似然函数（标称型数据））
        pClass1 -- 类别1的概率（先验概率）
    返回：
        判断结果 -- 0或1
    """
    #元素间乘法
    p1 = np.sum(vec2Classify * p1Vec) + np.log(pClass1)   
    p0 = np.sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)
    if p1 > p0:                                 #由于只需要比较相对大小，相同的p(W)也就不用除了
        return 1
    else: 
        return 0

In [11]:
def testingNB():                                     #对上述函数进行封装
    """
    测试朴素贝叶斯分类器
    参数：
        无
    返回：
        无 -- 直接打印结果
    """
    listOPosts,listClasses = loadDataSet()           #读取之前的数据
    myVocabList = createVocabList(listOPosts)        #产生词汇表
    trainMat=[]                                      #初始化一个训练矩阵
    for postinDoc in listOPosts:                     #遍历文档        
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))     #转化为向量后，添加到训练矩阵中
        #trainMat.append(bagOfWords2VecMN(myVocabList, postinDoc))  #词袋模型
    p0V,p1V,pAb = trainNB0(np.array(trainMat),np.array(listClasses))#得到三个概率
    
    testEntry = ['love', 'my', 'dalmation', 'garbage', 'garbage']                         #待测试文档
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))      #词组转为矩阵
    print("{} classified as: {}".format(testEntry, classifyNB(thisDoc,p0V,p1V,pAb))) #打印结果
    testEntry = ['stupid', 'garbage']                               #待测试词组                                
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))       #词组转为矩阵
    print("{} classified as: {}".format(testEntry, classifyNB(thisDoc,p0V,p1V,pAb))) #打印结果
testingNB()

['love', 'my', 'dalmation', 'garbage', 'garbage'] classified as: 0
['stupid', 'garbage'] classified as: 1


## 4.2.使用朴素贝叶斯过滤垃圾邮件（交叉验证测试算法）

In [18]:
def textParse(bigString):
    """
    输入很长的字符串，转换为向量
    参数：
        bigString -- 长字符串
    返回：
        去掉少于两个长度以下的字符，且转换为小写的字符串
    """
    import re
    listOfTokens = re.split(r'\W*', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2] 

In [35]:
def spamTest():
    """
    垃圾邮件测试
    参数：
        无
    返回：
        无
    """
    docList=[]; classList = []; fullText =[]                                     #新建三个列表
    for i in range(1,26):                                                        #遍历垃圾邮件和正常邮件，各25个       
        wordList = textParse(open("email/spam/{}.txt".format(i), errors = 'ignore').read())   #读取垃圾邮件
        docList.append(wordList)                                                              #添加到文档列表
        fullText.extend(wordList)                                                             #添加到类
        #extend方法只能接收list，且把这个list中的每个元素添加到原list中。
        #append方法可以接收任意数据类型的参数，并且简单地追加到list尾部。
        classList.append(1)                                                                   #读取正常邮件标记为1
                                                       #ham中的23.txt总是报错有不能解读的字节，选择忽略该错误
        wordList = textParse(open("email/ham/{}.txt".format(i), errors = 'ignore').read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)                                                #垃圾邮件标记为0 
    vocabList = createVocabList(docList)                                   #创建词汇表
    trainingSet = list(range(50)); testSet=[]                              #训练集和测试集序号集初始化
    for i in range(10):                                                     #随机抽取训练集中的10个序号，放入测试集
        randIndex = np.int(np.random.uniform(0,len(trainingSet)))          #生成随机序号   
        testSet.append(trainingSet[randIndex])                             #根据序号对应的元素由训练集移动到测试集中
        del(trainingSet[randIndex]) 
    trainMat=[]; trainClasses = []                                         #新建训练矩阵和训练标签   
    for docIndex in trainingSet:                                           #对于训练集中的元素
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))    #对应词袋添加到训练矩阵中
        trainClasses.append(classList[docIndex])                           #类别添加到标签中
    p0V,p1V,pSpam = trainNB0(np.array(trainMat),np.array(trainClasses))    #训练朴素贝叶斯分类器
    errorCount = 0                                                         #错误计数初始化为0
    for docIndex in testSet:                                               #遍历测试集
        #wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])        #得到词袋向量
        wordVector =  setOfWords2Vec(vocabList, docList[docIndex])
        print(classifyNB(np.array(wordVector),p0V,p1V,pSpam))
        if classifyNB(np.array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:   #比较模型预测结果与真实类别
            errorCount += 1                                                #统计错误
            print("错误序号为：{}".format(docIndex))                        #打印错误信息
    print("总准确率为：{}".format(1 - np.float(errorCount)/len(testSet)))
spamTest()
#多次执行，计算平均错误率即可完成交叉验证测试算法错判率

1
1
错误序号为：23
1
1
错误序号为：25
1
错误序号为：17
1
错误序号为：29
1
1
错误序号为：15
1
错误序号为：13
1
总准确率为：0.4
